# Creating channel maps

In [ ]:
import numpy as np
from ecogdata.channel_map import ChannelMap
import ecogdata.devices.electrode_pinouts as ep

(Basic `ChannelMap` mapping & imaging features are also seen in the [dataset demo](http://gabilan2.egr.duke.edu/labdocs/notebooks/dataset_demo_py3.html)).

## ChannelMap: channel-to-site lookups

Multichannel ephys recordings are usually conceptualized in data as Channels x Time (or the transpose). Every data point in the linear "channel space" is associated with an electrode site in the physical "array space" geometry. A `ChannelMap` is a channel-to-site lookup table for every ECoG channel in a dataset.

`ChannelMap` is implemented as a list (literally a subclass of Python list) of "flat" grid locations for each ECoG channel. Flat means that sites in a 3 x 4 matrix can be uniquely identified by numbers 0-11. Uniquely depends on convention. There is row-major (aka C-ordering from the C language) that counts up across columns and column-major (aka Fortran-order) that counts up across rows.

In [ ]:
print('fortran style')
print(np.arange(12).reshape(3, 4, order='F'))
print('c style')
print(np.arange(12).reshape(3, 4, order='C'))

*Note: row-major is more common outside of Matlab. The principal reason for accomodating both conventions is that multiplexed "active" array recordings are more naturally described using column-major ordering.*

Consider the first 3 channels of the large 256 Human array recorded through the Intan RHD DAQ.

In [ ]:
channel_map = ep.get_electrode_map('psv_256_rhd')[0]
print(channel_map[:3], 'grid size:', channel_map.geometry, 'col-major?', channel_map.col_major)

Since the map is row-major, the numbers count across 22 columns before wrapping around to the next row. The value "194" corresponds to 8 rows (or floor division with columns) and 18 columns (or remainder of division with columns).

In [ ]:
print('By hand:', 194 // 22, 194 % 22)
print('Lookup:', channel_map.rlookup(0))

In column-major, the flat index for this site would be instead 18 columns of 12 rows (18 * 12 = 216) plus 8 rows (216 + 8 = 224)

In [ ]:
print('Site (8, 18) in col-major:', channel_map.as_col_major()[0])

## Creating new mapping

Every channel mapping (except active array maps) is defined in the `ecogdata.devices.electrode_pinouts` module. I will attempt to describe different methods for mapping from crude to somewhat systematic. Available maps are listed in the "electrode_maps" dictionary.

In [ ]:
print('First few maps:', list(ep.electrode_maps.keys())[:5])

A few notes on "conventions" in this module:

* naming tends to go "(electrode)_(recording-system)"
* "intan" actually refers to open-ephys (we began using Intan chips with this DAQ)
* "rhd" signifies Intan chips with RHD DAQ
* "psv_61" was eventually replaced with "ratv(n)" for rat array revisions

In general, the module is a complete mess.. but still works.

Each entry in the `electrode_maps` dictionary has these entries:

* geometry: (rows, columns)
* rows: row index of each channel in DAQ order (including non-signal channels)
* cols: col index, as above
* pitch: inter-electrode pitch (if anisotropic, it can be (dx, dy))

For example, the ratv5 electrode recorded with open ephys 

* has 8 x 8 geometry
* rows and cols lists map all 64 DAQ channels
* the 3 "missing" channels (1, 31, 63 zero-based index) are coded as "grounded" (*more  on that later*)
* pitch is 420 micron

In [ ]:
print(ep.electrode_maps['ratv5_intan'])

### Method 1: hard-coded mapping
The easiest way to create such a mapping is to hard-code a channel-to-(row, col) mapping and give it a name that will age well. For example, the venerable 61-channel rat electrode hooked up to the 4x16 analog MUX headstage (so classic it's just "psv_61", like Prince).

**Note 1: all indices must be zero based, i.e. 0-7 inclusive for 8 rows / cols.**

**Note 2: electrode maps should be *pad down* to visualize LFP in the same orientation as one would look at cortex.** The electrode site-to-pin mappings are typically stated from the *pad-up* design perspective. Converting to pad-down simply means reversing the column index. Use the `_rev(num_columns, column_list)` method to simplify that task.

```python

psv_61 = dict(
    geometry=(8, 8),

    pitch=0.406,

    rows=[GND, 7, 7, 6, 6, 5, 5, 4, 4, 3, 3, 0, 3, 0, 3, 0, GND, 7, 6, 6,
          5, 5, 4, 4, 2, 3, 0, 3, 0, 3, 0, 3, GND, 7, 7, 6, 6, 5, 5, 4,
          4, 1, 2, 1, 2, 1, 2, 1, GND, 7, 6, 6, 5, 5, 4, 4, 1, 2, 1, 2,
          1, 2, 1, 2],

    cols=_rev(8, [GND, 3, 1, 3, 0, 3, 0, 3, 0, 0, 1, 1, 2, 2, 3, 3, GND,
                  6, 5, 6, 5, 6, 5, 6, 7, 7, 6, 6, 5, 5, 4, 4, GND, 5,
                  4, 7, 4, 7, 4, 7, 4, 7, 6, 6, 5, 5, 4, 4, GND, 2, 1,
                  2, 1, 2, 1, 2, 0, 0, 1, 1, 2, 2, 3, 3])

)

```

Then make sure this name is also listed in the `electrode_maps` dictionary.

#### Side note: GND, REF, and OTHER channels

The row and column maps seen above include `GND` or `<NonSignalChannels.grounded: 1>`. The words `GND`, `REF`, and `OTHER` are reserved for grounded-input (no connection), reference electrodes, or other non-signal channels. To add more channel types to this set, use the `Enum` defined near the beginning of the module.

### Method 2: hard-coded mapping with "A1" matrix coding

Another design convention lists electrode sites as "A1 A2 A3 ..." for 1st row (A) and columns 1-N. These coordinates are decoded to rows & columns using `unzip_encoded()`

**Note 1: certain letters are excluded from this alphabet ("IOQS", I believe), so H is the 8th row and J is the 9th row.**

**Note 2: in place of row-col, write in "REF" or "GND" for any reference/grounded channels.**

**Note 3: any *other* non-decodable string to code as OTHER (I like to use "~")**

```python
## Human 256 v2 (direct map from grid to Intan RHD channels)
h256_grid_by_channels = """J4, L4, G5, A8, D5, F7, C5, C8, F5, J8, D7, D9, H7, G8, H2, E9, H8, H4, K6, A5, E7, B5, 
H3, E5, G7, J1, L3, H5, L2, E6, B8, J3, J5, L7, J2, L6, C6, K7, B6, A7, F6, K2, A6, K4, K1, B7, K5, E8, G9, C7, L8, 
L1, B9, J7, C9, H6, K8, K3, L5, J6, F8, D6, D8, G6, K10, B10, L10, M8, M10, C10, L11, M6, K11, L9, F10, A9, M9, M2, 
E11, M1, M7, M11, H10, G11, D11, F11, G10, A11, J10, B11, E10, C11, D10, J11, A10, F3, H11, G1, F2, F4, C2, B4, E1, 
C4, D3, C3, D2, D4, D1, E2, E3, G3, H9, E4, M4, G2, J9, H1, F9, F1, K9, B3, G4, B2, M5, B1, M3, C1, F20, G19, F21, 
M20, F22, G21, B21, M18, B22, K14, E19, F14, H22, H14, E20, J14, M19, B20, C19, C20, E21, D22, D19, D21, B19, D20, 
F19, E22, G22, C22, G20, K13, C21, D13, L13, E13, H12, J13, C12, H13, B12, G12, A12, G13, F12, D12, E12, A13, M21, 
F13, M16, C13, M22, M14, A14, M13, L14, M12, B13, L12, M17, K12, M15, J12, J20, L18, J21, D15, H17, L22, J17, F15, 
D17, K15, C16, C14, J16, G14, K18, B14, L15, K20, A16, K21, B16, K22, K19, A17, K16, F17, L17, B17, L16, G17, E15, 
J19, C17, L21, G18, L20, J18, G16, H18, K17, J22, A18, E18, H20, B18, E16, H21, B15, G15, D16, H15, F16, E14, H16, 
D14, D18, J15, H19, L19, C18, C15, F18, A15, E17"""
h256_rows_cols = list(unzip_encoded(h256_grid_by_channels))
h256_rows_cols[1] = _rev(22, h256_rows_cols[1])
psv_256_rhd = {
    'rows': h256_rows_cols[0],
    'cols': h256_rows_cols[1],
    'pitch': 1.72,
    'geometry': (12, 22)
}
```

### Method 3: interconnecting pinouts

A better way to specify maps is by connecting each component electrode-pin mapping. This is done in a number of cases using the `connect_passive_map` method.

In this system, an electrode map is the combination of 

1. a pinout map of the electrode array to its own connectors
2. any potential inter-connecting maps
3. the order of connector pins read out in DAQ channels

Take the "rat reference electrode" as a case study. Start by defining the map of ZIF pins to electrode locations. The electrode locations can be written out as (row, column) entries, or encoded as "A1, A2, ..." as before. In this case, the string encoding is used and then decoded into (row, column) pairs. 

#### Electrode side
First, the electrode locations are written in the order of ZIF pins 1-61.

In [ ]:
rat_refelectrode_by_zif = """REF, H3, H4, G3, G1, G2, G4, F3, F1, F2, F4, E3, E1, E2, E4, D1, B1, A1, C1, D2, B2, A2, 
C2, D3, B3, A3, C3, D4, B4, A4, C4, C5, A5, B5, D5, C6, A6, B6, D6, C7, A7, B7, D7, C8, B8, D8, E5, E7, E8, E6, F5, 
F7, F8, F6, G5, G7, G8, G6, H5, H6, REF"""
# make it a static list so we can print a couple entries without spending up the zip generator
rat_refelectrode_by_zif_rc = list(zip(*ep.unzip_encoded(rat_refelectrode_by_zif)))
print(rat_refelectrode_by_zif_rc[:5])

The basic electrode map is formed as a lookup table from ZIF pin name (here just numbers 1-61) to electrode array locations. **This map is inherent to the electrode and does not change with different DAQ hardware.**

In [ ]:
rat_refelectrode_by_zif_lut = dict(zip(range(1, 62), rat_refelectrode_by_zif_rc))
print('\n'.join(['ZIF {}: site {}'.format(n, rat_refelectrode_by_zif_lut[n]) for n in range(1, 6)]))

#### Data side
To make a channel map for a particular recording combination, the rat reference electrode ZIF map can be hooked up to different DAQs. Just need to list the order that ZIF pins are sampled for a particular DAQ. The data end of the channel map is specified by writing the appropriate pin keys (ZIF pin numbers here) in the order that they are sampled by the DAQ. For open ephys or RHD sampling, ZIF pins are read-out in this order (the `None` value is used if there is no pin on that data channel).

(*Big caveat -- in this example, the order of ZIF pins in the Intan channels depends on the panasonic P4 interconnect that is not explicitly written out.*)

In [ ]:
zif_by_intan64 = [32, None, 34, 48, 36, 50, 38, 52, 40, 54, 42, 56, 44, 58,
                  46, 60, 47, 49, 45, 51, 43, 53, 41, 55, 39, 57, 37, 59, 35,
                  61, 33, None, 31, 1, 29, 3, 27, 5, 25, 7, 23, 9, 21,
                  11, 19, 13, 17, 15, 16, 2, 18, 4, 20, 6, 22, 8, 24, 10,
                  26, 12, 28, 14, 30, None]

#### Connecting maps
Electrode-side maps and data-side maps are connected (with any pin-changing interconnect maps) using `connect_passive_map`, which returns a mapping dictionary as seen in method 1.

In [ ]:
help(ep.connect_passive_map)

In [ ]:
map_dict = ep.connect_passive_map((8, 8), rat_refelectrode_by_zif_lut, zif_by_intan64, pitch=0.4)
print(map_dict)

To make a map for the same electrode on the MUX v6 headstage & NI DAQ, we only change the order that the ZIF pins are sampled:

In [ ]:
zif_by_mux6 = [None, 1, 2, 4, 6, 8, 10, 12, 14, 16, 20, 22, 24, 26, 28, 30,
               None, 60, 58, 56, 54, 52, 50, 48, 46, 44, 42, 40, 38, 36, 34, 32,
               None, 61, 59, 57, 55, 53, 51, 49, 47, 45, 43, 41, 39, 37, 35, 33,
               None, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]
map_dict = ep.connect_passive_map((8, 8), rat_refelectrode_by_zif_lut, zif_by_mux6, pitch=0.4)
print(map_dict)

These maps are made available in the `electrode_pinouts` module as in this example:

```python

electrode_maps = dict(
    # new passive map construction
    ratv5_intan=connect_passive_map((8, 8), rat_v5_by_zif_lut,
                                    zif_by_intan64, pitch=0.420),
    ...
)
```